<a href="https://colab.research.google.com/github/kimtomato/FL_KCN-project/blob/master/VKN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.6.0


In [2]:
!pip uninstall keras

Found existing installation: keras 2.6.0
Uninstalling keras-2.6.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/keras-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/keras/*
Proceed (y/n)? y
  Successfully uninstalled keras-2.6.0


In [9]:
!pip install tensorflow==2.5.0
!pip install tensorflow_federated==0.19.0
!pip install keras==2.5.0rc0
!pip install tensorflow_privacy==0.7.3

     |████████████████████████████████| 251 kB 10.7 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 4.0 MB 52.3 MB/s 
  Attempting uninstall: attrs
    Found existing installation: attrs 19.3.0
    Uninstalling attrs-19.3.0:
      Successfully uninstalled attrs-19.3.0
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
  Attempting uninstall: tensorflow-privacy
    Found existing installation: tensorflow-privacy 0.5.2
    Uninstalling tensorflow-privacy-0.5.2:
      Successfully uninstalled tensorflow-privacy-0.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-federated 0.19.0 requires attrs~=19.3.0, but you have attrs 21.2.0 which is incompatibl

In [11]:
!pip install --quiet --upgrade nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#fed.py
import collections
import random

import sys

import tensorflow as tf
import tensorflow_federated as tff

NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10


def preprocess(dataset):

    def batch_format_fn(element):
        """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
        return collections.OrderedDict(
            x=tf.reshape(element['pixels'], [-1, 784]),
            y=tf.reshape(element['label'], [-1, 1]))

    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


def make_federated_data(client_data, client_ids):
    return [
        preprocess(client_data.create_tf_dataset_for_client(x))
        for x in client_ids
    ]


def create_keras_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(784,)),
        tf.keras.layers.Dense(10, kernel_initializer='zeros'),
        tf.keras.layers.Softmax(),
    ])


emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
preprocessed_example_dataset = preprocess(example_dataset)


def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_dataset.element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


evaluation = tff.learning.build_federated_evaluation(model_fn)


class FLData:

    def __init__(self):
        self.emnist_train, self.emnist_test = tff.simulation.datasets.emnist.load_data()

    def get_nb_samples(self):
        return len(self.emnist_train.client_ids)

    def get_data_samples_id(self, nb_samples):
        return random.choices(self.emnist_train.client_ids, k=nb_samples)

    def get_federated_data(self, sample_ids):
        return make_federated_data(self.emnist_train, sample_ids)

    def get_federated_test_data(self, sample_ids):
        return make_federated_data(self.emnist_test, sample_ids)

    def get_all_samples_id(self):
        return self.emnist_train.client_ids

    def get_all_test_samples_id(self):
        return self.emnist_test.client_ids


class FLDataSelector:

    def __init__(self, worlddim, mode, fl_data, cut):

        self.div = cut
        if mode == "localized":
            print('SELECTOR: {}x{} cells'.format(cut, cut))

        self.dim = float(worlddim)
        self.fl_data = fl_data
        if mode == "random" or mode == "localized":
            self.mode = mode
        else:
            self.mode = "random"

        self.slices = None
        if mode == "localized":
            # Generate location-based data.
            size = self.div
            self.slices = []
            for i in range(size):
                self.slices.append(self.fl_data.get_data_samples_id(size))

    def get_dataslice(self, x, y):

        if self.mode == "localized":

            x_ = min(max(x, 0), self.dim - 0.01)
            y_ = min(max(y, 0), self.dim - 0.01)

            #print (x_,y_)

            i = int(x_ / self.dim * self.div)
            j = int(y_ / self.dim * self.div)

            #print (i,j)
            #print (self.slices[i][j])
            return self.slices[i][j]

        else:
            return self.fl_data.get_data_samples_id(1)[0]


class FLTraining:
    @staticmethod
    def set_seed(seed):
        tf.random.set_seed(seed)

    def __init__(self, validation_set):

        self.iterative_process = tff.learning.build_federated_averaging_process(
            model_fn, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(
                learning_rate=0.02), server_optimizer_fn=lambda: tf.keras.optimizers.SGD(
                learning_rate=1.0))

        self.state = self.iterative_process.initialize()
        self.validation_set = validation_set
        self.round_id = 0

        self.current_evaluation = None

    def training_round(self, federated_data):

        if len(federated_data) == 0:
            print('round {:2d}, skip.'.format(self.round_id))

        else:
            self.state, _ = self.iterative_process.next(
                self.state, federated_data)
            self.current_evaluation = evaluation(
                self.state.model, self.validation_set)
            print('round {:2d}, eval_metrics={}'.format(
                self.round_id, self.current_evaluation))

        print("size:{}".format(sys.getsizeof(self.state.model)))

        self.round_id += 1
        return self.current_evaluation

'''
NUM_CLIENTS = 10
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))


logdir = "logs/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()


sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(emnist_train, sample_clients)
print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
if len(federated_train_data) > 0:
    print('First dataset: {d}'.format(d=federated_train_data[0]))


evaluation = tff.learning.build_federated_evaluation(model_fn)

x = []
y1 = []
Y1 = []
y2 = []
Y2 = []


NUM_ROUNDS = 80
#with summary_writer.as_default():
for round_num in range(1, NUM_ROUNDS):

    clients_set = random.choices(emnist_train.client_ids, k=10)
    print("round {} clients:{}".format(round_num, clients_set))
    federated_train_data = make_federated_data(emnist_train, clients_set)
    federated_test_data = make_federated_data(emnist_test, emnist_train.client_ids[0:50])

    state, metrics = iterative_process.next(state, federated_train_data)

    eval_metrics = evaluation(state.model, federated_test_data)
    print('round {:2d}, eval_metrics={}'.format(round_num, eval_metrics))

    x.append(round_num)
    y1.append(eval_metrics['loss'])
    Y1.append(eval_metrics['sparse_categorical_accuracy'])

    #for name, value in dict(eval_metrics).items():
    #    tf.summary.scalar(name, value, step=round_num)

    #print('round {:2d}, metrics={}'.format(round_num, metrics))


state = iterative_process.initialize()

for round_num in range(1, NUM_ROUNDS):

    clients_set = random.choices(emnist_train.client_ids, k=random.randint(1,10))
    print("round {} clients:{}".format(round_num, clients_set))
    federated_train_data = make_federated_data(emnist_train, clients_set)
    federated_test_data = make_federated_data(emnist_test, emnist_train.client_ids[0:50])

    state, metrics = iterative_process.next(state, federated_train_data)

    eval_metrics = evaluation(state.model, federated_test_data)
    print('round {:2d}, eval_metrics={}'.format(round_num, eval_metrics))

    y2.append(eval_metrics['loss'])
    Y2.append(eval_metrics['sparse_categorical_accuracy'])

    #for name, value in dict(eval_metrics).items():
    #    tf.summary.scalar(name, value, step=round_num)

    #print('round {:2d}, metrics={}'.format(round_num, metrics))



plt.title('Loss')
plt.xlabel('round number')
plt.ylabel('loss')
plt.plot(x,y1,label='10 clients')
plt.plot(x,y2, label='2 clients')
plt.legend()
plt.show()

plt.title('Acc')
plt.xlabel('round number')
plt.ylabel('Acc')
plt.plot(x,Y1,label='10 clients')
plt.plot(x,Y2, label='2 clients')
plt.legend()
plt.show()
'''

In [ ]:
#Mobility.py
import numpy as np

PI = 3.14159


class World:
    """A simple class holding information about the size of the area containing all the
    vehicles and helper methods to compute points and distances."""

    def __init__(self, dimension):
        self.dimension = dimension

    def random_point(self):
        return np.random.uniform(high=self.dimension, size=2)

    @staticmethod
    def random_point_within_circle(center, radius):
        r = np.random.uniform() * radius
        angle = np.random.uniform() * 2 * PI

        return center + r * np.array((np.cos(angle), np.sin(angle)))

    @staticmethod
    def distance(a, b):
        return np.linalg.norm(a - b)

    @staticmethod
    def intermediate_point(a, b, distance):
        (dy, dx) = (b[1] - a[1], b[0] - a[0])
        angle = np.math.atan2(dy, dx)

        return a + distance * np.array((np.cos(angle), np.sin(angle)))


class MovingNode:
    """ A class representing an abstract moving node. The vehicle is able to move within the bounds
    described by a World instance, following a (steady-state) Random Waypoint mobility model."""

    def __init__(self, world, node_id):
        self.n_id = node_id
        self.position = None

    def __eq__(self, other):
        return (self.n_id == other.n_id)

    def move(self, timestep):
        pass


class RWPNode(MovingNode):
    def __init__(self, world, node_id):
        self.n_id = node_id
        self.position = None
        self.world = world

        self.pause_probability = 0.5
        self.pause_max = 300

        self.min_speed = 5
        self.max_speed = 20

        self._init_initial_path()

    def __eq__(self, other):
        return (self.n_id == other.n_id)

    def _init_initial_path(self):
        # Will the node begin in a paused state ?
        u_pause = np.random.uniform()
        if u_pause < self.pause_probability:

            u = np.random.uniform()
            self.current_pause_time = self.pause_max * (1 - np.sqrt(1 - u))
            self.position = np.random.uniform(size=2) * self.world.dimension
            self._destination = self.position  # Will be recomputed at the end of the pause
            self._speed = 0

            #print ('node id {}: starting in a pause of {}s at pos {}'.format(self.n_id, self.current_pause_time, self.position))

        else:  # The node begins in movement

            self.current_pause_time = 0

            # Initial speed (steady-state)
            u = np.random.uniform()
            self._speed = (self.max_speed**u) / (self.min_speed**(u - 1))

            # Initial path (steady-state)
            path_chosen = False
            while not path_chosen:
                (x1, y1) = np.random.uniform(size=2)
                (x2, y2) = np.random.uniform(size=2)

                r = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) / np.sqrt(2)
                u1 = np.random.uniform()

                if u1 < r:
                    u2 = np.random.uniform()

                    self.position = np.array(
                        (u2 * x1 + (1 - u2) * x2, u2 * y1 + (1 - u2) * y2)) * self.world.dimension
                    self._destination = np.array(
                        (x2, y2)) * self.world.dimension
                    path_chosen = True

            #print ("node id {}: starting in move: pos:{}, dest:{}, speed:{}".format(self.n_id, self.position, self._destination, self._speed))

    def _compute_new_pause(self):

        #print ("node id {}: computing new potential pause ?".format(self.n_id))
        u_pause = np.random.uniform()

        if u_pause < self.pause_probability:
            u_pausetime = np.random.uniform()
            self.current_pause_time = u_pausetime * self.pause_max
            #print ("nid {}: there will be a pause of {}s".format(self.n_id, self.current_pause_time))

        else:  # No pause: compute new move
            self.current_pause_time = 0
            #print ("nid {}: there will be no pause".format(self.n_id))
            self._compute_new_path()

    def _compute_new_path(self):

        # Path
        self.position = self._destination
        self._destination = np.random.uniform(size=2) * self.world.dimension

        # Speed
        u = np.random.uniform()
        self._speed = self.min_speed + u * (self.max_speed - self.min_speed)

        #print ("nid {}: computed new path: current pos:{}, dest:{}, speed:{}".format(self.n_id, self.position, self._destination, self._speed))

    def move(self, timestep):

        # State: pause ?
        if self.current_pause_time > 0:
            self.current_pause_time = np.maximum(
                0, self.current_pause_time - timestep)
            #print ("nid {}: pausing at pos {}... remaining pause after this step:{}".format(self.n_id, self.position, self.current_pause_time))

            if self.current_pause_time == 0:  # Pause finished
                self._compute_new_path()
            return

        # State: driving
        distance_to_dest = World.distance(self.position, self._destination)
        driven_distance = self._speed * float(timestep)

        #print ("driven distance:{}, dist to dest:{}".format(driven_distance, distance_to_dest))

        if driven_distance >= distance_to_dest:
            # print("recomputing")
            self.position = self._destination
            #print ("nid {} arrived at destination {}={}".format(self.n_id, self.position, self._destination))

            self._compute_new_pause()

        else:  # Continue the drive to waypoint
            self.position = World.intermediate_point(
                self.position, self._destination, driven_distance)
            #print ("nid {}: driving: pos:{}, dest:{}, new dist to dest:{}".format(self.n_id, self.position, self._destination, World.distance(self.position, self._destination)))

    # Helper function: useful for eventual RPGM Following nodes.

    def get_reference_point_area(self):
        return {'pos': self.position, 'radius': 25}

    def get_max_speed(self):
        return self.max_speed

    def is_pausing(self):
        return (self.current_pause_time > 0)

    def get_angle(self):
        a = self.position
        b = self._destination

        (dy, dx) = (b[1] - a[1], b[0] - a[0])
        return np.math.atan2(dy, dx)


class RPGMFollowingNode(MovingNode):
    ''' Node that follows a group leader (the group leader follows steady-state RWP) '''

    def __init__(self, leader, node_id):
        self.n_id = node_id
        self.leader = None

        self.reference_point = None
        self.ref_dist = None
        self.set_group_leader(leader)

    def __eq__(self, other):
        return (self.n_id == other.n_id)

    def set_group_leader(self, leader):
        self.leader = leader
        ref = self.leader.get_reference_point_area()

        # Set initial position
        self.reference_point = World.random_point_within_circle(ref['pos'], 25)
        self.ref_dist = (
            self.reference_point[0] -
            self.leader.position[0],
            self.reference_point[1] -
            self.leader.position[1])

        self.position = World.random_point_within_circle(
            self.reference_point, 25)

    def move(self, timestep):

        # Follow the leader
        if self.leader.is_pausing():
            return

        # Compute new position based on speed and angle
        self.reference_point = np.array(
            [self.leader.position[0] + self.ref_dist[0], self.leader.position[1] + self.ref_dist[1]])
        self.position = World.random_point_within_circle(
            self.reference_point, 25)

        # self.update_speed_angle()
        #self.position += timestep*self.speed*np.array((np.cos(self.angle), np.sin(self.angle)))

        #ref = self.leader.get_reference_point()
        #self.position = World.random_point_within_circle(self.leader.position, 50)

        #print ('v{}, speed:{}, angle:{}'.format(self.n_id, self.speed, self.angle))


# vehicles_count must be divisible by groups_count
def generate_rpgm_mobility(groups_list, world):

    nodes = []

    i = 0
    for group_size in groups_list:

        leader = RWPNode(world, i)
        nodes.append(leader)
        #print("leader {}".format(i))

        for _ in range(group_size - 1):
            i += 1
            follower = RPGMFollowingNode(leader, i)
            nodes.append(follower)
            #print("follower {}".format(i))

        i += 1

    return nodes


'''
world = World(1000)
nodes = generate_rpgm_mobility(50, 5, world)


timestep = 2
for i in range(0,20):
    for n in nodes:
        n.move(timestep)

    pt_leaders = []
    pt_followers = []
    for n in nodes:
        if isinstance(n, RPGMFollowingNode):
            pt_followers.append(n.position)
        else:
            pt_leaders.append(n.position)

    pt_leaders = np.array(pt_leaders)
    pt_followers = np.array(pt_followers)

    plt.scatter(pt_followers[:, 0], pt_followers[:, 1], color='blue')
    plt.scatter(pt_leaders[:, 0], pt_leaders[:, 1], color='orange', marker='x')
    plt.xlim((0,world.dimension))
    plt.ylim((0,world.dimension))
    plt.show()


world = World(1000)
leader = RWPNode(world, 1)

v1 = RPGMFollowingNode(leader, 2)
v2 = RPGMFollowingNode(leader, 3)
v3 = RPGMFollowingNode(leader, 4)


timestep = 2
for i in range(0,100000):
    leader.move(timestep)
    v1.move(timestep)
    v2.move(timestep)
    v3.move(timestep)

point_leader = np.array([leader.position])
vehs = np.array([v1.position, v2.position, v3.position])

plt.scatter(vehs[:, 0], vehs[:, 1], color='blue')
plt.scatter(point_leader[:, 0], point_leader[:, 1], color='orange', marker='x')
plt.xlim((0,world.dimension))
plt.ylim((0,world.dimension))
plt.show()
'''

"\nworld = World(1000)\nnodes = generate_rpgm_mobility(50, 5, world)\n\n\ntimestep = 2\nfor i in range(0,20):\n    for n in nodes:\n        n.move(timestep)\n\n    pt_leaders = []\n    pt_followers = []\n    for n in nodes:\n        if isinstance(n, RPGMFollowingNode):\n            pt_followers.append(n.position)\n        else:\n            pt_leaders.append(n.position)\n\n    pt_leaders = np.array(pt_leaders)\n    pt_followers = np.array(pt_followers)\n\n    plt.scatter(pt_followers[:, 0], pt_followers[:, 1], color='blue')\n    plt.scatter(pt_leaders[:, 0], pt_leaders[:, 1], color='orange', marker='x')\n    plt.xlim((0,world.dimension))\n    plt.ylim((0,world.dimension))\n    plt.show()\n\n\nworld = World(1000)\nleader = RWPNode(world, 1)\n\nv1 = RPGMFollowingNode(leader, 2)\nv2 = RPGMFollowingNode(leader, 3)\nv3 = RPGMFollowingNode(leader, 4)\n\n\ntimestep = 2\nfor i in range(0,100000):\n    leader.move(timestep)\n    v1.move(timestep)\n    v2.move(timestep)\n    v3.move(timestep)\n\

In [ ]:
#simulation.py
import operator
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

class RandEvent:
    """A class representing a randomly generated event, as well as
    static methods to generate a timeline of events following a
    Poisson process."""

    def __init__(self):
        self.center = None
        self.radius = None
        self.time = None
        self.duration = None
        self.data_slice = None

    def current(self, time):
        if self.time is None or self.duration is None:
            return False

        return self.time <= time and time <= self.time + self.duration

    # Returns a data item from the event's data slice
    def observe_data(self):
        if self.data_slice is None:
            return None

        return self.data_slice

    def __str__(self):
        return "center:{}, radius:{}, time:{}, duration:{}, data_slice:{}".format(
            self.center, self.radius, self.time, self.duration, self.data_slice)

    @staticmethod
    def generate_events_timeline(
            timelimit,
            rate,
            ev_size,
            ev_duration,
            world,
            fed_data_selector):
        # Poisson process
        events = []
        time = 0

        max_duration = 0

        while time < timelimit:
            next_time = -np.log(1.0 - np.random.uniform()) / rate
            time += next_time

            # Generate event
            next_event = RandEvent()
            next_event.center = world.random_point()
            next_event.radius = np.random.normal(loc=ev_size, scale=1.0)
            next_event.duration = np.random.normal(loc=ev_duration, scale=10.0)
            next_event.data_slice = fed_data_selector.get_dataslice(
                next_event.center[0], next_event.center[1])
            next_event.time = time

            if (next_event.duration > max_duration):
                max_duration = next_event.duration

            events.append((time, next_event))

        return (events, max_duration)

    # Returns whether "vehicle" matches "event"
    @staticmethod
    def match_event(vehicle, event, time):
        return event.current(time) and World.distance(
            vehicle.mobility.position, event.center) <= event.radius

    # Returns whether "vehicle" matches at least one event from "events"

    @staticmethod
    def match_events(vehicle, events, time):
        for e in events:
            if RandEvent.match_event(vehicle, e, time):
                #print ("match! vid:{}, vpos:{}, vspeed:{} e:{}".format(vehicle.v_id, vehicle.position, vehicle._speed, e))
                return e

        return None

    # Finds the index right before (if smallerIndex == True) or after "value"
    # in "l[start:end]"
    @staticmethod
    def find_ix_sorted(l, value, start, end, smaller_index):

        if start == end - 1:
            return start if smaller_index else end

        mid_point = (start + end) // 2
        if l[mid_point] == value:
            return mid_point
        elif l[mid_point] < value:
            return RandEvent.find_ix_sorted(
                l, value, mid_point, end, smaller_index)
        elif l[mid_point] > value:
            return RandEvent.find_ix_sorted(
                l, value, start, mid_point, smaller_index)

    # Return the list of events happening between start_time and end_time
    # (excluded)
    @staticmethod
    def select_events_between(events, start_time, end_time):

        times = list((time for (time, _) in events))

        begin = RandEvent.find_ix_sorted(
            times, start_time, 0, len(times), True)
        end = RandEvent.find_ix_sorted(times, end_time, 0, len(times), False)

        return events[begin + 1:end]


DB_DISCARD_TIME = 300
DB_DISCARD_THRESHOLD = 500


class Vehicle:
    """ A class representing a vehicle. The vehicle is able to move within the bounds
    described by a World instance, following a (steady-state) Random Waypoint mobility model."""

    def __init__(self, mobility_model, v_id):

        self.v_id = v_id
        self.mobility = mobility_model

        # "Database"
        self.db_last_obs = None

    def __eq__(self, other):
        return (self.v_id == other.v_id)

    def drive(self, timestep):
        self.mobility.move(timestep)

    # Discard old data
    def update_db(self, time):

        if self.db_last_obs is not None:

            # First condition: timeout?
            if self.db_last_obs['time'] + DB_DISCARD_TIME < time:
                self.db_last_obs = None

            # Second condition: out of relevant area?
            elif World.distance(self.mobility.position, self.db_last_obs['position']) > DB_DISCARD_THRESHOLD:
                #print("vid {}: discard interest because of distance:{}".format(self.v_id, World.distance(self.position, self.db_last_obs[1])))
                self.db_last_obs = None

    def rx_model_forstep(self, step_id, network_tx, time):

        #print ("vid {} rx model".format(self.v_id))
        if self.db_last_obs is None:
            print(
                "vid {}: discarding model for step id {} received at time {}, no data".format(
                    self.v_id, step_id, time))
            return

        localtraining_time = np.random.normal(loc=1, scale=0.5)
        network_tx.to_coordinator(
            step_id,
            self.v_id,
            time + localtraining_time,
            self.db_last_obs['federated_data'])
        print(
            "vid {}: model for step id {} received from coordinator at time {}, training and sending back at time {}".format(
                self.v_id,
                step_id,
                time,
                time +
                localtraining_time))

    def observe(self, time, event, network_tx):
        #print ("vid {}, observation at time {} and loc {}".format(self.v_id, time, self.position))

        observed_data = event.observe_data()
        if observed_data is None:
            return

        position_cpy = np.copy(self.mobility.position)
        self.db_last_obs = {
            "time": time,
            "position": position_cpy,
            "federated_data": observed_data}
        network_tx.notify_interest_to_coordinator(
            self.v_id, position_cpy, time, time)

    def observe_with_selector(self, time, fed_selector, network_tx):
        #print ("vid {}, observation at time {} and loc {}".format(self.v_id, time, self.position))

        if self.db_last_obs is None or self.db_last_obs['time'] + 10 <= time:

            position_cpy = np.copy(self.mobility.position)
            self.db_last_obs = {
                "time": time,
                "position": position_cpy,
                "federated_data": fed_selector.get_dataslice(
                    position_cpy[0],
                    position_cpy[1])}
            network_tx.notify_interest_to_coordinator(
                self.v_id, position_cpy, time, time)

    @staticmethod
    def generate_vehicles_RWP(nb_vehicles, world):
        vehicles = []
        for i in range(nb_vehicles):
            vehicles.append(Vehicle(RWPNode(world, i), i))

        return vehicles

    @staticmethod
    def generate_vehicles_RPGM(groups, world):
        vehicles = []
        mobility_nodes = generate_rpgm_mobility(groups, world)

        for node in mobility_nodes:
            vehicles.append(Vehicle(node, node.n_id))

        return vehicles


class NetworkTx:
    """ A simple class to simulate transmission delays of messages between
    the federated learning coordinator and the vehicles.
    Messages to be sent are cached and passed on after a timeout."""

    def __init__(self):

        self.tx_to_vehicle = []
        self.tx_to_coordinator = []
        self.tx_interest_to_coordinator = []

    def notify_interest_to_coordinator(
            self, v_id, location, sense_time, transfer_time):

        tx_time = np.random.normal(loc=1.0, scale=1.0)
        self.tx_interest_to_coordinator.append(
            (transfer_time + tx_time, v_id, location, sense_time))

    # Vehicle transmits model update to coordinator (from step "step_id") at
    # time "transfer_time"
    def to_coordinator(self, step_id, v_id, transfer_time, federated_data):

        tx_time = np.random.normal(loc=1.0, scale=1.0)
        self.tx_to_coordinator.append(
            (transfer_time + tx_time, v_id, step_id, federated_data))

    # Coordinator transmits model for training step "step_id" to "vehicle" at
    # time "transfer_time"
    # Coordinator transmits model (step_id) to "vehicle"
    def to_vehicle(self, vehicle, step_id, transfer_time):

        tx_time = np.random.normal(loc=1.0, scale=1.0)
        self.tx_to_vehicle.append((transfer_time + tx_time, vehicle, step_id))

    def update(self, coordinator, time):
        # Tx coordinator -> vehicle
        to_keep_tovehicle = []
        for (rx_time, vehicle, step_id) in self.tx_to_vehicle:
            if time >= rx_time:
                vehicle.rx_model_forstep(step_id, self, time)
                #print ("time {}, vid {} received model for step {}".format(time, vehicle.v_id, step_id))
            else:
                to_keep_tovehicle.append((rx_time, vehicle, step_id))
        self.tx_to_vehicle = to_keep_tovehicle

        # Tx vehicle -> coordinator
        to_keep_tocoordinator = []
        for (rx_time, v_id, step_id, federated_data) in self.tx_to_coordinator:
            if time >= rx_time:
                coordinator.rx_model_update(
                    v_id, step_id, time, federated_data)
                #print ("time {}, coordinator received update from vid {} for step {}".format(time, v_id, step_id))
            else:
                to_keep_tocoordinator.append(
                    (rx_time, v_id, step_id, federated_data))
        self.tx_to_coordinator = to_keep_tocoordinator

        # Tx vehicle interest -> coordinator
        to_keep_interests = []
        for (
            rx_time,
            v_id,
            location,
                interest_time) in self.tx_interest_to_coordinator:
            if time >= rx_time:
                coordinator.rx_vehicle_interest(
                    v_id, location, interest_time, time)
                #print ("time {}, coordinator received model interest: vid {} time of sensing {}".format(time, v_id, interest_time))
            else:
                to_keep_interests.append(
                    (rx_time, v_id, location, interest_time))
        self.tx_interest_to_coordinator = to_keep_interests


INTEREST_DISCARDTIME = 300


class Coordinator:
    """ A class representing the FL coordinator in charge for training a model
    based on the EMNIST dataset by the FedAvg algorithm. The coordinator initiates
    a new training step every T=15s, in which a set of vehicles is chosen to receive
    and locally train the current model. The coordinator then waits and receives model
    updates back from replying vehicles, before aggregating the results.
    Additionally, the coordinator receives training interests from vehicles,
    that it uses for pertinent client selection in the VKN-assisted approach."""

    def __init__(
            self,
            veh_per_step,
            fed_training,
            fed_data_handler,
            statistics):

        self.statistics = statistics
        self.fed_training = fed_training
        self.fed_data_handler = fed_data_handler

        self.nb_updates_total = 0
        self.modelsent_step = []
        self.updatesreceived_step = []
        self.step_fed_data = []

        self.step_duration = 15
        self.vehicles_per_step = veh_per_step['veh_per_step']
        self.vkn_adapt_vperstep = veh_per_step['vkn_adapt_vperstep']

        self.step_id = -1
        self.step_starttime = -21

        self.vkn_known_interests = {}

    def update(self, vehicles, time, network_tx, vkn):
        if time > self.step_starttime + self.step_duration:

            if self.step_id != -1:

                # Perform federated learning, aggregation of received models at
                # last step.
                print(
                    "step {}: fed data: {}".format(
                        self.step_id,
                        self.step_fed_data))
                step_eval = self.fed_training.training_round(
                    self.fed_data_handler.get_federated_data(self.step_fed_data))

                # Statistics
                if step_eval is not None:
                    self.statistics.register_training_evaluation(
                        self.step_id, step_eval['sparse_categorical_accuracy'],step_eval['loss'])

                self.statistics.register_step_efficiency(self.step_id, len(
                    self.updatesreceived_step) / self.vehicles_per_step)
                print("Last step: {}/{} models OK".format(
                    len(self.updatesreceived_step), self.vehicles_per_step))

                #print ("Known interests count: {}".format(len(self.vkn_known_interests.keys())))

                # Remove vehicles that did not respond from the interests list
                #print ("self.modelsent_step:{}".format(self.modelsent_step))
                #print ("self.updatesreceived_step:{}".format(self.updatesreceived_step))
                vehicles_noresponse = [
                    v_id for v_id in self.modelsent_step if v_id not in self.updatesreceived_step]
                print("Unresponsive vehicles: {}".format(vehicles_noresponse))

                new_interests = {}
                for (v_id, (interest_time, position)
                     ) in self.vkn_known_interests.items():
                    if v_id not in self.modelsent_step or v_id in self.updatesreceived_step:
                        new_interests[v_id] = (interest_time, position)
                        #print("Keeping interest {}".format(v_id))

                self.vkn_known_interests = new_interests

            self.step_fed_data = []
            self.modelsent_step = []
            self.updatesreceived_step = []

            # Interests update
            interests_to_keep = {}
            for (v_id, (interest_time, position)
                 ) in self.vkn_known_interests.items():
                if interest_time + INTEREST_DISCARDTIME >= time:
                    interests_to_keep[v_id] = (interest_time, position)
            self.vkn_known_interests = interests_to_keep

            if vkn and len(self.vkn_known_interests.keys()) > 0:
                print("Coordinator: new step (vkn) (time={})".format(time))
                self.new_step_vkn(vehicles, time, network_tx)
            else:
                print("Coordinator: new step (time={})".format(time))
                self.new_step_random(vehicles, time, network_tx)

    def new_step_vkn(self, vehicles, time, network_tx):

        self.step_id += 1
        self.step_starttime = time

        selected_indexes = []

        # From interests
        #print ( "interests:{}".format( np.array(list(self.vkn_known_interests.keys())) ) )

        sorted_interests = sorted(
            self.vkn_known_interests.items(),
            key=operator.itemgetter(1),
            reverse=True)
        sorted_vids = list((v_id for (v_id, _) in sorted_interests))

        selected_indexes_interests = np.array(
            sorted_vids[0:np.minimum(len(sorted_vids), self.vehicles_per_step)])
        #selected_indexes_interests = np.array(list(self.vkn_known_interests.keys()))[np.random.choice(len(self.vkn_known_interests), size=np.minimum(len(self.vkn_known_interests), self.vehicles_per_step), replace=False)]
        #print ( "chosen interests: vids: {}".format(selected_indexes_interests) )

        for v_id in selected_indexes_interests:
            selected_indexes.append(v_id)

        nb_random_vehicles = self.vehicles_per_step - len(selected_indexes)

        selected_indexes_tmp = []
        selected_interest_pos = []

        # LESS KNOWN INTERESTS THAN CLIENTS TO SELECT: SELECT ALL INTERESTS +
        # FILL WITH RANDOM
        if len(self.vkn_known_interests.items()) <= self.vehicles_per_step:
            for vid in self.vkn_known_interests.keys():
                selected_indexes.append(vid)

            nb_random_vehicles = self.vehicles_per_step - len(selected_indexes)
            if nb_random_vehicles > 0:
                remaining_indexes = np.array([ix for ix in range(
                    len(vehicles)) if ix not in selected_indexes_interests])
                extra_selected_indexes = remaining_indexes[np.random.choice(
                    len(remaining_indexes), size=nb_random_vehicles, replace=False)]

                for v_id in extra_selected_indexes:
                    selected_indexes.append(v_id)

        else:  # Enough interests: CLUSTERING OF INTERESTS

            interests_vid = []
            interests_times = []
            interests_locations = []
            for (vid, (interest_time, loc)
                 ) in self.vkn_known_interests.items():
                interests_vid.append(vid)
                interests_times.append(interest_time)
                interests_locations.append(loc)

            interests_vid = np.array(interests_vid)
            interests_times = np.array(interests_times)
            interests_locations = np.array(interests_locations)

            kmeans = KMeans(
                n_clusters=self.vehicles_per_step,
                tol=10,
                max_iter=30)
            kmeans.fit(interests_locations)

            # Taking the most recent vehicles by cluster.
            for cluster_id in range(self.vehicles_per_step):
                print("cluster {}".format(cluster_id))

                cluster_point_ids = []
                for j in range(len(interests_vid)):
                    if kmeans.labels_[j] == cluster_id:
                        cluster_point_ids.append(j)

                cluster_point_ids = np.array(cluster_point_ids)
                newest_interest = -1
                newest_time = -1
                newest_location = -1
                for point_id in cluster_point_ids:

                    if newest_interest == - \
                            1 or interests_times[point_id] > newest_time:
                        newest_interest = interests_vid[point_id]
                        newest_time = interests_times[point_id]
                        newest_location = interests_locations[point_id]

                '''locs = interests_locations[cluster_point_ids]
                print (locs)

                plt.scatter(locs[:, 0], locs[:, 1], color='orange')
                plt.scatter(kmeans.cluster_centers_[cluster_id,0], kmeans.cluster_centers_[cluster_id,1], color='green')
                plt.xlim((0,1000))
                plt.ylim((0,1000))
                plt.show()'''

                selected_interest_pos.append(newest_location)
                selected_indexes_tmp.append(newest_interest)

            # Final step: Reduce amount of clients selected if enabled
            remove_indexes = []
            if self.vkn_adapt_vperstep:
                for (i, pos1) in enumerate(selected_interest_pos):

                    finished = False
                    j = i + 1
                    while j < len(selected_interest_pos) and not finished:
                        pos2 = selected_interest_pos[j]
                        if World.distance(pos1, pos2) <= 50:
                            remove_indexes.append(i)
                            finished = True
                        j += 1

            selected_indexes = []
            for (ix, value) in enumerate(selected_indexes_tmp):
                if ix not in remove_indexes:
                    selected_indexes.append(value)

            print("NB CLIENTS SELECTED: {}".format(len(selected_indexes)))

            '''
            debug = np.array(vehicles)[np.array(selected_indexes)]
            selected_vehicles_pos = []
            for v in debug:
                selected_vehicles_pos.append(v.mobility.position)
            selected_vehicles_pos = np.array(selected_vehicles_pos)

            all_vehicles_pos = []
            for v in vehicles:
                all_vehicles_pos.append(v.mobility.position)
            all_vehicles_pos = np.array(all_vehicles_pos)

            random_vehs = np.random.choice(vehicles, 10)
            random_vehicles_pos = []
            for v in random_vehs:
                random_vehicles_pos.append(v.mobility.position)
            random_vehicles_pos = np.array(random_vehicles_pos)

            plt.title("Clustering of training samples' location of sensing by the coordinator")
            plt.scatter(all_vehicles_pos[:, 0], all_vehicles_pos[:, 1], color='gray', alpha=0.3) # All vehicles
            plt.scatter(random_vehicles_pos[:,0], random_vehicles_pos[:,1], color='orange', marker='x') # Random selected v
            plt.scatter(selected_vehicles_pos[:,0], selected_vehicles_pos[:,1], color='blue', marker='x') # VKN selected v
            plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], color='purple', marker='$c$') # Clusters
            plt.xlim((0,1000))
            plt.ylim((0,1000))
            plt.show()'''

        #print ("final selected indexes: selected vids:{}".format(selected_indexes))

        self.statistics.register_step_nbselect(
            self.step_id, len(selected_indexes))
        selected_vehicles = np.array(vehicles)[np.array(selected_indexes)]

        print("step_id:{}, sent to vehicles:".format(self.step_id))

        for v in selected_vehicles:
            network_tx.to_vehicle(v, self.step_id, time)
            print(v.v_id)

        self.modelsent_step = selected_indexes

    def new_step_random(self, vehicles, time, network_tx):

        self.step_id += 1
        self.step_starttime = time

        selected_indexes = np.random.choice(
            len(vehicles), size=self.vehicles_per_step, replace=False)
        selected_vehicles = np.array(vehicles)[selected_indexes]
        print("step_id:{}, sent to vehicles:".format(self.step_id))

        for v in selected_vehicles:
            network_tx.to_vehicle(v, self.step_id, time)
            print(v.v_id)

        self.modelsent_step = selected_indexes

    # Reception of a model update from vehicle
    def rx_model_update(self, v_id, step_id, time, federated_data):
        if step_id != self.step_id:  # An old model, discard
            print(
                "time {} vid {} stepid {} != current step id {}: Model discarded".format(
                    time, v_id, step_id, self.step_id))
            return

        print(
            "Model update received at time {}, vid:{} / stepid:{} ; federated data id: {}".format(
                time,
                v_id,
                step_id,
                federated_data))
        self.step_fed_data.append(federated_data)
        self.nb_updates_total += 1
        self.updatesreceived_step.append(v_id)

        self.statistics.register_new_step(time, v_id)

    def rx_vehicle_interest(self, v_id, location, interest_time, rx_time):

        #print ("time {}: Coordinator received interest (vid:{}, location: {}, interest time:{})".format(rx_time, v_id, location, interest_time))
        self.vkn_known_interests[v_id] = (
            interest_time, [location[0], location[1]])


class Statistics:
    """ A simple class to keep track of the progress of a simulation."""

    def __init__(self):
        self.steps_per_time = {}
        self.step_efficiency = {}
        self.step_nbselect = {}

        self.training_accuracy = {}
        self.training_loss = {}

        self.participating_vids = []
        self.newvids_per_time = {}

    def register_new_step(self, time, v_id):

        if time not in self.newvids_per_time:
            self.newvids_per_time[time] = 0

        if v_id not in self.participating_vids:
            self.participating_vids.append(v_id)
            self.newvids_per_time[time] += 1

        if time not in self.steps_per_time:
            self.steps_per_time[time] = 0

        self.steps_per_time[time] += 1

    def register_step_efficiency(self, step_id, rate):
        self.step_efficiency[step_id] = rate

    def register_step_nbselect(self, step_id, nbselect):
        self.step_nbselect[step_id] = nbselect

    def register_training_evaluation(self, step_id, accuracy, loss):
        self.training_accuracy[step_id] = accuracy
        self.training_loss[step_id] = loss

    def get_total_steps_per_time(self):

        # Total steps per time. [0] (time) and [1] (aggregated steps)
        times = []
        total_steps = []

        current_total = 0
        for time in sorted(self.steps_per_time.keys()):
            current_total += self.steps_per_time[time]

            total_steps.append(current_total)
            times.append(time)

        return (times, total_steps)

    def get_cumulated_newvids_per_time(self):

        # Total steps per time. [0] (time) and [1] (aggregated steps)
        times = []
        total_vids = []

        current_total = 0
        for time in sorted(self.newvids_per_time.keys()):
            current_total += self.newvids_per_time[time]

            total_vids.append(current_total)
            times.append(time)

        return (times, total_vids)

    def plot_total_steps_per_time(self):

        (times, total_steps) = self.get_total_steps_per_time()

        plt.plot(times, total_steps)
        plt.show()

    # Smoothed dict {time:value}

    def get_smoothed_list(self, data, smooth):
        x = []
        y = []

        times = list(sorted(data.keys()))

        smooth_value = 0
        smooth_index = 0
        for timeval in times:

            smooth_value += data[timeval]
            smooth_index += 1

            if smooth_index == smooth:
                x.append(timeval)
                y.append(smooth_value / smooth)
                smooth_index = 0
                smooth_value = 0

        return (x, y)

    def get_step_efficiency(self, smooth):
        return self.get_smoothed_list(self.step_efficiency, smooth)

    def get_step_nbselect(self, smooth):
        return self.get_smoothed_list(self.step_nbselect, smooth)

    def get_training_accuracy(self, smooth):
        return self.get_smoothed_list(self.training_accuracy, smooth)

    def get_training_loss(self, smooth):
        return self.get_smoothed_list(self.training_loss, smooth)

    def plot_step_efficiency(self, smooth):
        (x, y) = self.get_step_efficiency(smooth)
        plt.plot(x, y)
        plt.show()

    def plot_training_accuracy(self, smooth):
        (x, y) = self.get_training_accuracy(smooth)
        plt.title("Training accuracy")
        plt.plot(x, y)
        plt.show()

    def plot_training_loss(self, smooth):
        (x, y) = self.get_training_loss(smooth)
        plt.title("Training loss")
        plt.plot(x, y)
        plt.show()


class Simulation:
    """ A class to initialize and run a simulation."""

    def __init__(
            self,
            size,
            nb_vehicles,
            mobility,
            event_conf,
            timelimit,
            timestep,
            veh_per_step,
            vkn,
            seed):

        np.random.seed(seed)
        FLTraining.set_seed(seed)

        self.time = 0
        self.timelimit = timelimit
        self.timestep = timestep
        self.vkn = vkn
        self.veh_per_step = veh_per_step

        self.world = World(size)

        self.vehicles = []
        if mobility['model'] == "RPGM" and 'groups' in mobility:
            self.vehicles = Vehicle.generate_vehicles_RPGM(
                mobility['groups'], self.world)
            print("Using RPGM model: v:{} groups:{}".format(
                sum(mobility['groups']), mobility['groups']))

        else:  # Random Waypoint
            self.vehicles = Vehicle.generate_vehicles_RWP(
                nb_vehicles, self.world)

        self.fed_data = FLData()
        self.fed_training = FLTraining(
            self.fed_data.get_federated_test_data(
                self.fed_data.get_all_test_samples_id()[
                    0:20]))

        # Events generation
        use_events = event_conf['use_events']
        event_data_distribution = event_conf['data_distribution']
        cut = 0
        if 'cut' in event_conf:
            cut = event_conf['cut']

        self.fed_data_selector = FLDataSelector(
            self.world.dimension, event_data_distribution, self.fed_data, cut)
        (self.events, self.events_max_duration) = (None, None)

        if use_events:
            event_rate = event_conf['rate']
            event_size = event_conf['size']
            event_duration = event_conf['duration']
            (self.events, self.events_max_duration) = RandEvent.generate_events_timeline(
                self.timelimit, event_rate, event_size, event_duration, self.world, self.fed_data_selector)

        self.statistics = Statistics()
        self.coordinator = Coordinator(
            self.veh_per_step,
            self.fed_training,
            self.fed_data,
            self.statistics)
        self.network_tx = NetworkTx()

    def step(self):

        self.time += self.timestep

        # Currently happening events
        current_events = None
        if self.events is not None:
            current_events = RandEvent.select_events_between(
                self.events, self.time - self.events_max_duration - 0.01, self.time + 0.01)

        # Drive vehicles
        for v in self.vehicles:
            v.drive(self.timestep)
            v.update_db(self.time)

            # Match events
            if current_events is None:
                v.observe_with_selector(
                    self.time, self.fed_data_selector, self.network_tx)
            else:
                ev = RandEvent.match_events(
                    v, (ev for (_, ev) in current_events), self.time)
                if ev is not None:
                    v.observe(self.time, ev, self.network_tx)

        #print ("time: {}".format(self.time))
        self.coordinator.update(
            self.vehicles,
            self.time,
            self.network_tx,
            vkn=self.vkn)
        self.network_tx.update(self.coordinator, self.time)

        return self.time

    def done(self):
        return self.time >= self.timelimit

In [ ]:
#rud_fed.py
import os
import sys
#import argparse
import easydict      #jupyter에서서 argparse라이브러리리 사용 시 에러발생
import pickle
import nest_asyncio
nest_asyncio.apply()


class NoStdStreams(object):
    def __init__(self, stdout=None, stderr=None):
        self.devnull = open(os.devnull, 'w')
        self._stdout = stdout or self.devnull or sys.stdout
        self._stderr = stderr or self.devnull or sys.stderr

    def __enter__(self):
        self.old_stdout, self.old_stderr = sys.stdout, sys.stderr
        self.old_stdout.flush()
        self.old_stderr.flush()
        sys.stdout, sys.stderr = self._stdout, self._stderr

    def __exit__(self, exc_type, exc_value, traceback):
        self._stdout.flush()
        self._stderr.flush()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr
        self.devnull.close()


# parser = argparse.ArgumentParser(description='Run FL/VKN simulations.')
# parser.add_argument(
#     '--mobility',
#     default="RWP",
#     help='="RWP" or "RPGM". Generate simulation results using Random Waypoint or RPGM mobility.')
# parser.add_argument(
#     '--seed_begin',
#     type=int,
#     default=0,
#     help='Simulations will be generated for random seeds [seed_begin;seed_end[.')
# parser.add_argument(
#     '--seed_end',
#     type=int,
#     default=1,
#     help='Simulations will be generated for random seeds [seed_begin;seed_end[.')

# args = parser.parse_args()

args = easydict.EasyDict({
    "mobility" : "RWP",
    "seed_begin" : 0,
    "seed_end" : 1
})





# Simulation's input parameters
events_conf = {}
mob = {}
training = {}
mobility_name = ""
vkn_adapt_vperstep = False

vps = 10  # Number of training vehicles per step

if args.mobility == "RWP":

    events_conf = {
        "use_events": True,
        "rate": 1.5 / 60.0,
        "size": 50.0,
        "duration": 60.0,
        "data_distribution": "random"}
    mob = {"model": 'RWP'}
    training = {"veh_per_step": 10, "vkn_adapt_vperstep": False}

    mobility_name = "rwp"
    vkn_adapt_vperstep = False

else:
    vkn_adapt_vperstep = True
    events_conf = {
        "use_events": False,
        "data_distribution": "localized",
        "cut": 10}
    mob = {"model": 'RPGM', "groups": [650, 20, 20, 10, 10, 10, 10, 10, 5, 5]}
    training = {"veh_per_step": vps, "vkn_adapt_vperstep": True}

    mobility_name = "rpgm"
    vkn_adapt_vperstep = True


stats_vkn = {}
stats_tradi = {}

stats_vkn[vps] = []
stats_tradi[vps] = []


for i in range(args.seed_begin, args.seed_end):

    print("New simulations with seed: {}".format(i))

    simvkn = Simulation(
        size=1000,
        nb_vehicles=750,
        mobility=mob,
        event_conf=events_conf,
        timelimit=3600,
        timestep=1,
        veh_per_step=training,
        vkn=True,
        seed=i)
    simtradi = Simulation(
        size=1000,
        nb_vehicles=750,
        mobility=mob,
        event_conf=events_conf,
        timelimit=3600,
        timestep=1,
        veh_per_step=training,
        vkn=False,
        seed=i)

    print("\tVKN simulation...")
    while not simvkn.done():
        with NoStdStreams():
            time = simvkn.step()
        print("time: {}/3600s".format(time), end="\r")
    stats_vkn[vps].append(simvkn.statistics)

    print("\tvkn ok")
    print("\tTraditional simulation...")
    while not simtradi.done():
        with NoStdStreams():
            time = simtradi.step()
        print("time: {}/3600s".format(time), end="\r")
    stats_tradi[vps].append(simtradi.statistics)


prefix = ""
if vkn_adapt_vperstep:
    prefix = "optimized_"

stats = {'vkn': stats_vkn, 'tradi': stats_tradi}
filename = "{}{}_dump{}_{}_{}".format(
    prefix,
    mobility_name,
    vps,
    args.seed_begin,
    args.seed_end)
with open(filename, 'wb') as filehandler:
    pickle.dump(stats, filehandler)

print('done.')

New simulations with seed: 0
	VKN simulation...


KeyError: ignored